In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow_hub as hub
from tensorflow import keras

In [1]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 31.9 MB/s eta 0:00:00


In [3]:
import keras_core as keras
import keras_nlp

Using TensorFlow backend


In [5]:
test = pd.read_csv("drive/MyDrive/Tweet/tweet_test.csv")
tw = pd.read_csv("drive/MyDrive/Tweet/tweet_train.csv")
tw

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
X = tw["text"]
y = tw["target"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset("distil_bert_base_en_uncased", sequence_length=160, name ="preprocessor_for_tweets")

100%|██████████| 580/580 [00:00<00:00, 403kB/s]
100%|██████████| 226k/226k [00:00<00:00, 305kB/s]


In [9]:
classifier = keras_nlp.models.DistilBertClassifier.from_preset("distil_bert_base_en_uncased", num_classes=2, preprocessor=preprocessor)

100%|██████████| 515/515 [00:00<00:00, 327kB/s]
100%|██████████| 253M/253M [00:17<00:00, 15.3MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [10]:
classifier.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_test, y_test))

Epoch 1/2
191/191 [==============================] - 157s 614ms/step - loss: 0.4412 - sparse_categorical_accuracy: 0.8054 - val_loss: 0.4226 - val_sparse_categorical_accuracy: 0.8135
Epoch 2/2
191/191 [==============================] - 101s 528ms/step - loss: 0.3191 - sparse_categorical_accuracy: 0.8752 - val_loss: 0.4332 - val_sparse_categorical_accuracy: 0.8168


In [15]:
classifier.compile(
      loss = tf.keras.losses.CategoricalCrossentropy(),
      optimizer = tf.keras.optimizers.Adam(),
      metrics = ["accuracy"])

In [16]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [17]:
testx = test["text"]

In [18]:
f = classifier.predict(testx)

102/102 [==============================] - 22s 193ms/step


In [19]:
fn = np.argmax(f, axis=1)

In [20]:
final = pd.DataFrame()
final["id"] = test["id"]
final["target"] = fn
final

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,0


In [21]:
final["target"].value_counts()

0    2384
1     879
Name: target, dtype: int64

In [22]:
final.to_csv("drive/MyDrive/Tweet/tweet_pred2.csv", index=False)